In [3]:
import pandas as pd
import requests
import time

def get_lat_lon_from_address(address, api_key):
    """
    카카오맵 API를 사용해 주소를 위도와 경도로 변환하는 함수
    
    Args:
        address (str): 변환할 주소 (도로명 또는 지번)
        api_key (str): 카카오 디벨로퍼스에서 발급받은 REST API 키

    Returns:
        tuple: (위도, 경도) 튜플. 실패 시 (None, None) 반환.
    """
    KAKAO_API_URL = "https://dapi.kakao.com/v2/local/search/address.json"
    
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    
    params = {
        "query": address
    }
    
    try:
        response = requests.get(KAKAO_API_URL, headers=headers, params=params, timeout=5)
        response.raise_for_status() # 오류 발생 시 예외 발생
        
        data = response.json()
        
        # 'documents' 리스트가 비어있지 않고, 첫 번째 결과가 있는지 확인
        if data.get('documents'):
            # 카카오 API는 위도를 'y', 경도를 'x'로 반환합니다.
            latitude = data['documents'][0]['y']
            longitude = data['documents'][0]['x']
            return (latitude, longitude)
        else:
            print(f"주소 변환 실패 (결과 없음): {address}")
            return (None, None)
            
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {address} ({e})")
        return (None, None)
    except Exception as e:
        print(f"기타 오류: {address} ({e})")
        return (None, None)



In [ ]:
from dotenv import load_dotenv
import time # API 서버 부하를 줄이기 위해 time 라이브러리 추가
import os
# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("kakao_reset_key") 
# 1. 파일 읽기
df = pd.read_csv("../data_csv/11_10/춘천시_요식업소현황_2504.csv", encoding='cp949') 



# 3. 위도, 경도 변환 적용
# '소재지(도로명)' 컬럼을 기준으로 변환합니다. 
# 만약 도로명 주소가 비어있는 경우 '소재지(지번)'을 사용하도록 할 수도 있습니다.
# 여기서는 '소재지(도로명)'을 우선 사용합니다.
# 참고: 데이터가 많을 경우 시간이 매우 오래 걸릴 수 있습니다.

# 결과를 저장할 리스트 초기화
latitudes = []
longitudes = []

print("주소 변환을 시작합니다. 데이터 양에 따라 시간이 오래 걸릴 수 있습니다...")

for index, row in df.iterrows():
    address_to_search = row['소재지(도로명)']
    
    # 만약 '소재지(도로명)'이 비어있다면 '소재지(지번)' 사용 (선택적)
    if pd.isna(address_to_search) or address_to_search.strip() == "":
        address_to_search = row['소재지(지번)']
        
    if pd.isna(address_to_search):
        lat, lon = None, None
    else:
        # API 무료 할당량(쿼터) 초과를 방지하기 위해 약간의 지연 시간(sleep)을 줍니다.
        # time.sleep(0.01) # 1초에 100회 미만으로 조절
        
        lat, lon = get_lat_lon_from_address(address_to_search, api_key)

    latitudes.append(lat)
    longitudes.append(lon)
    
    if (index + 1) % 100 == 0:
        print(f"{index + 1} / {len(df)} 건 처리 중...")

# 4. DataFrame에 새로운 컬럼 추가
df['위도'] = latitudes
df['경도'] = longitudes

print("주소 변환 완료!")

# 5. 결과 파일로 저장
output_filename = "../data_csv/11_10/춘천시_요식업소_좌표포함.csv"
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"'{output_filename}' 파일로 저장이 완료되었습니다.")
print(df[['업소명', '소재지(도로명)', '위도', '경도']].head())

주소 변환을 시작합니다. 데이터 양에 따라 시간이 오래 걸릴 수 있습니다...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 소양로3가 180-6 
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 춘천로81번길 16, 1,2층 (약사동)
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 북산면 신샘밭로 1152, 1~2층
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 금강로62번길 7, 1,2층 (조양동)
100 / 6272 건 처리 중...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 명동길29번길 22 (죽림동)
200 / 6272 건 처리 중...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 영서로 2875, 1,2층 (사농동)
300 / 6272 건 처리 중...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 명동길29번길 8-4, 8-2 1층 (죽림동)
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 남산면 서천길 159-1
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 서면 경춘로 1517-5 (2리)
400 / 6272 건 처리 중...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 남산면 서천길 7
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 서면 삿갓봉길 20
500 / 6272 건 처리 중...
600 / 6272 건 처리 중...
700 / 6272 건 처리 중...
800 / 6272 건 처리 중...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 우묵길74번길 14, 1,2,3층 (퇴계동)
900 / 6272 건 처리 중...
1000 / 6272 건 처리 중...
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 동면 연산골길 54, 1,2동
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 남산면 북한강변길 688-79
1100 / 6272 건 처리 중...


In [7]:
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv

def get_lat_lon_from_address(address, api_key):
    """
    카카오맵 API를 사용해 주소를 위도와 경도로 변환하는 함수
    
    Args:
        address (str): 변환할 주소 (도로명 또는 지번)
        api_key (str): 카카오 디벨로퍼스에서 발급받은 REST API 키

    Returns:
        tuple: (위도, 경도) 튜플. 실패 시 (None, None) 반환.
    """
    KAKAO_API_URL = "https://dapi.kakao.com/v2/local/search/address.json"
    
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    
    params = {
        "query": address
    }
    
    try:
        response = requests.get(KAKAO_API_URL, headers=headers, params=params, timeout=5)
        response.raise_for_status() # 오류 발생 시 예외 발생
        
        data = response.json()
        
        if data.get('documents'):
            latitude = data['documents'][0]['y']
            longitude = data['documents'][0]['x']
            return (latitude, longitude)
        else:
            print(f"주소 변환 실패 (결과 없음): {address}")
            return (None, None)
            
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {address} ({e})")
        return (None, None)
    except Exception as e:
        print(f"기타 오류: {address} ({e})")
        return (None, None)

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("kakao_reset_key") 

# 1. 파일 읽기
# (사용자님이 제공한 파일 경로 대신, 업로드된 파일 이름을 사용하도록 수정했습니다.)
df = pd.read_csv("../data_csv/11_10/춘천시_요식업소현황_2504.csv", encoding='cp949') 


# ★★★★★
# 2. 100건의 샘플 데이터 생성
# 원본 df 대신 100개짜리 df_sample을 사용합니다.
# .copy()를 붙여야 경고 메시지(SettingWithCopyWarning)가 뜨지 않습니다.
df_sample = df.head(100).copy() 
# ★★★★★


# 3. 위도, 경도 변환 적용 (100건에 대해서만)
latitudes = []
longitudes = []

print(f"주소 변환을 시작합니다. (총 {len(df_sample)}건 샘플 테스트)")

# ★★★★★
# 원본 df.iterrows() 대신 df_sample.iterrows()로 변경
for index, row in df_sample.iterrows():
    address_to_search = row['소재지(도로명)']
    
    if pd.isna(address_to_search) or address_to_search.strip() == "":
        address_to_search = row['소재지(지번)']
        
    if pd.isna(address_to_search):
        lat, lon = None, None
    else:
        # time.sleep(0.01) # 1초에 100회 미만으로 조절
        
        lat, lon = get_lat_lon_from_address(address_to_search, api_key)

    latitudes.append(lat)
    longitudes.append(lon)
    
    if (index + 1) % 50 == 0: # 100건이므로 50건마다 표시하도록 변경
        print(f"{index + 1} / {len(df_sample)} 건 처리 중...")

# ★★★★★
# 4. DataFrame에 새로운 컬럼 추가 (df_sample에 추가)
df_sample['위도'] = latitudes
df_sample['경도'] = longitudes

print("주소 변환 완료!")

# 5. 결과 파일로 저장
output_filename = "../data_csv/11_10/춘천시_요식업소_좌표포함(100건_샘플).csv" # 파일 이름 변경
df_sample.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"'{output_filename}' 파일로 저장이 완료되었습니다.")
print(df_sample[['업소명', '소재지(도로명)', '위도', '경도']].head())

주소 변환을 시작합니다. (총 100건 샘플 테스트)
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 소양로3가 180-6 
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 춘천로81번길 16, 1,2층 (약사동)
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 북산면 신샘밭로 1152, 1~2층
주소 변환 실패 (결과 없음): 강원특별자치도 춘천시 금강로62번길 7, 1,2층 (조양동)
50 / 100 건 처리 중...
100 / 100 건 처리 중...
주소 변환 완료!
'../data_csv/11_10/춘천시_요식업소_좌표포함(100건_샘플).csv' 파일로 저장이 완료되었습니다.
          업소명                          소재지(도로명)                위도  \
0      춘천세종호텔         강원특별자치도 춘천시 봉의산길 31 (봉의동)  37.8866252853171   
1        강촌식당            강원특별자치도 춘천시 남산면 강촌로 56  37.8138534737228   
2         중화루  강원특별자치도 춘천시 둥지길 5-17, 2,3층 (효자동)  37.8755454305588   
3         대화관  강원특별자치도 춘천시 서부대성로44번길 12-2 (요선동)  37.8822884820124   
4  옥산포코다리(춘천)        강원특별자치도 춘천시 영서로 3048 (사농동)  37.9241849100018   

                 경도  
0  127.731541748401  
1    127.6331260179  
2  127.735770180448  
3   127.72729626171  
4  127.721350665355  
